In [1]:
from __future__ import annotations
import polars as pl
import inspect
import numpy as np
from typing import Tuple
import dsds.metrics as me
import dsds.prescreen as ps
import dsds.sample as sa
import dsds.fs as fs

In [5]:
import dsds.transform as t
df = pl.DataFrame({
    "a": [-2, 2, 3]
})
t.log_transform(df, cols=["a"], plus_one=True)

a,a_log1p
i64,f64
-2,NaN
2,1.098612
3,1.386294


In [ ]:
df = pl.read_csv("../data/advertising.csv")
df.head()

In [ ]:
%%timeit
df.lazy().select(["City", "Country", "Clicked on Ad"]).groupby(1).agg(
            pl.all().unique().sort()
        ).select(["City", "Country", "Clicked on Ad"]).collect()

In [ ]:
%%timeit
df.unique(subset=["City", "Country", "Clicked on Ad"]).select(["City", "Country", "Clicked on Ad"])

In [ ]:
%%timeit
fs.discrete_ig(df, target="Clicked on Ad")

In [ ]:
%%timeit 
fs.discrete_ig2(df, target="Clicked on Ad")

In [ ]:
df = pl.DataFrame({
    "a": [1, np.nan, None],
    "b": [1,2,3]
})
df

In [ ]:
ps.invalid_inferral(df, threshold=0.5, include_null=True)

In [ ]:
ps.discrete_inferral(df)

In [ ]:
temp = ps.get_unique_count(df.with_row_count(offset=1).set_sorted("row_nr"), include_null_count=True)
len_df = temp.filter(pl.col("column") == "row_nr").item(0,1)
print(len_df)
temp

In [ ]:
pred = np.random.random(size=1_000_000) # .reshape(-1, 1) # Predictions
actual = np.round(np.random.random(size=1_000_000)).astype(np.int8)

In [ ]:
actual

In [ ]:
from sklearn.datasets import make_classification
orig_x, orig_y = make_classification(n_samples = 50_000, n_features = 50, n_informative = 20, n_redundant = 30)
df_pl = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))
features = df_pl.columns
features.remove("target")

In [ ]:
%%timeit
fs.discrete_ig(df_pl, target="target", cols=features)

In [ ]:
%%timeit
fs.discrete_ig2(df_pl, target="target", cols=features)

In [ ]:

me.logloss(actual, pred, check_binary=False)